<a href="https://colab.research.google.com/github/ZichangYe/NBA-Analytics/blob/master/Q2_v1015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports 

In [3]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F 
from IPython.display import Image 
from torchvision import transforms
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 12345
random.seed(seed)
torch.manual_seed(seed)

#### Mount your google drive so you can save model checkpoints, and report your test results on the final best model after hyperparameter tuning

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data loading 

##### Run the following cells to load the dataset. Setting download=True will download it for you.

In [5]:
cifar10_train = torchvision.datasets.CIFAR10(root='./cifar10', 
                                             train=True, 
                                             transform=None, 
                                             target_transform=None,
                                             download=True)
cifar10_test = torchvision.datasets.CIFAR10(root='./cifar10', 
                                             train=False, 
                                             transform=None, 
                                             target_transform=None,
                                             download=True)

# Divides the dataset into train and val so that we can use the val to choose our hyperparameters
train_dataset, val_dataset = torch.utils.data.random_split(cifar10_train, [40000, 10000], 
                                                           generator=torch.Generator().manual_seed(12345))
test_dataset = cifar10_test

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


In [6]:
# Helper code to support adding different transforms on the dataset lazily after downloading the dataset
# From https://discuss.pytorch.org/t/apply-different-transform-data-augmentation-to-train-and-validation/63580/5
class MapDataset(torch.utils.data.Dataset):
    """
    Given a dataset, creates a dataset which applies a mapping function
    to its items (lazily, only when an item is called).

    Note that data is not cloned/copied from the initial dataset.
    """

    def __init__(self, dataset, map_fn):
        self.dataset = dataset
        self.map = map_fn

    def __getitem__(self, index):
        if self.map:     
            x = self.map(self.dataset[index][0]) 
        else:     
            x = self.dataset[index][0]  
        y = self.dataset[index][1]         
        return x, y

    def __len__(self):
        return len(self.dataset)

#### Standard transforms to apply on images - Convert to tensors and normalize with mean and std. These are the basic transforms that you will always apply. The mean and std have been pre calculated on the training set. 

In [7]:
# Notice that we apply the same mean and std normalization calculated on train, to both the train and test datasets.
test_transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         [0.4373, 0.4434, 0.4725],
                                         [0.1201, 0.1231, 0.1052])
                                     ])

train_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.4373, 0.4434, 0.4725],
                                          [0.1201, 0.1231, 0.1052])
                                      ])


In [8]:
train_dataset_w_transform  = MapDataset(train_dataset, train_transform)
val_dataset_w_transform = MapDataset(val_dataset, test_transform)
test_dataset_w_transform = MapDataset(test_dataset, test_transform)

In [9]:
bs = 128
torch.backends.cudnn.benchmark = True
train_loader = DataLoader(train_dataset_w_transform, batch_size=bs, shuffle=True, drop_last=False,num_workers=10, pin_memory=True)
val_loader = DataLoader(val_dataset_w_transform, batch_size=bs, shuffle=False, drop_last=False,num_workers=10, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False, drop_last=False,num_workers=10, pin_memory=True)

### Q 2.1 Training loop

In this question, fill in the missing parts to build a generic training loop that returns the train and validation losses and accuracies. The #TODOs will guide you through the key points and you should fill some code for each #TODO. You might need to add some additional code for bookkeeping the losses and accuracies

In [10]:
def train_loop(model, criterion, optimizer,  train_loader, val_loader):
    """
    Generic training loop

    Parameters
    ----------
    model : Object instance of your model class 
    criterion : Loss function 
    optimizer : Instance of optimizer class of your choice 
    train_loader : Training data loader 
    val_loader : Validation data loader

    Returns
    -------
    train_losses : List with train loss on dataset per epoch
    train_accuracies : List with train accuracy on dataset per epoch
    val_losses : List with validation loss on dataset per epoch
    val_accuracies : List with validation accuracy on dataset per epoch

    """
    best_val = 0.0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    max_patience = 5
    patience_counter = 0
    # Training
    for t in tqdm(range(50)):
        # TODO : Set the model to train mode        
        model.train()
        train_correct = 0
        val_correct = 0
        # TODO: Loop over the training set 
        for batch_idx, (data, target) in enumerate(train_loader):
            # TODO: Put the inputs and targets on the write device
            data, target = data.to(device), target.to(device)
            # TODO: Feed forward to get the logits
            output = model(data)
            # TODO: Compute the loss and accuracy
            train_loss = criterion(output, target)
            train_pred = output.data.max(1, keepdim=True)[1]
            train_correct += train_pred.eq(target.data.view_as(train_pred)).cpu().sum().item()
            train_accuracy = 100. * train_correct/len(train_loader.dataset)
            # print('Training Accuracy: {}'.format(train_accuracy))
            # TODO: zero the gradients before running
            optimizer.zero_grad()
            # the backward pass.
            
            # TODO: Backward pass to compute the gradient
            # of loss w.r.t our learnable params. 
            train_loss.backward()
            # TODO: Update params
            optimizer.step()
            # TODO: Keep track of accuracy and loss
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
        # TODO: Switch the model to eval mode
        model.eval()

        with torch.no_grad():
            # TODO: Loop over the validation set 
            for batch_idx, (data, target) in enumerate(val_loader):
                # TODO: Put the inputs and targets on the write device
                data, target = data.to(device), target.to(device)
                # TODO: Feed forward to get the logits
                output = model(data)
                # TODO: Compute the loss and accuracy
                val_loss = criterion(output, target)
                val_pred = output.data.max(1, keepdim=True)[1]
                val_correct += val_pred.eq(target.data.view_as(val_pred)).cpu().sum().item()
                val_accuracy = 100. * val_correct/len(val_loader.dataset)
                # print('Validation Accuracy: {}'.format(val_accuracy))
     
                # TODO: Keep track of accuracy and loss
            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)              
        if val_accuracies[-1] > best_val:
          best_val = val_accuracies[-1]
          patience_counter = 0

          # TODO: Save best model, optimizer, epoch_number
          best_model = model
          best_optimizer = optimizer
          best_epoch_number = t
        else:
          patience_counter += 1    
          if patience_counter > max_patience: 
            break

        print("[EPOCH]: %i, [TRAIN LOSS]: %.6f, [TRAIN ACCURACY]: %.3f" % (t, train_losses[-1], train_accuracies[-1]))
        print("[EPOCH]: %i, [VAL LOSS]: %.6f, [VAL ACCURACY]: %.3f \n" % (t, val_losses[-1] ,val_accuracies[-1]))

    return train_losses, train_accuracies, val_losses, val_accuracies

### Q 2.2 Shallow convolutional network. 

In [11]:
class View(nn.Module):
    def __init__(self, shape):
      super().__init__()
      self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

In [12]:
ShallowNet =  nn.Sequential(
      nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size=5, padding=2),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=8),
      View((-1,256)),
      nn.Linear(256, 10),
)

#### Write the object oriented version of ShallowNet

In [43]:
class ShallowConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        """

        Parameters
        ----------
        input_channels : Number of input channels
        num_classes : Number of classes for the final prediction 
        """
        super(ShallowConvnet, self).__init__()
        self.input_channels = input_channels
        # TODO
        self.network = nn.Sequential(
            nn.Conv2d(in_channels = input_channels, out_channels = 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=8),
            View((-1,256)),
            nn.Linear(256, num_classes),
      )
        

    def forward(self, x):
        """

        Parameters
        ----------
        x

        Returns
        -------
        output : Result after running through the model
        """
        # TODO
        # x = x.view(-1, self.input_channels)?
        return self.network(x)
        

### Q2.3 Instantiate the model and run this using an SGD optimizer, with the appropriate loss function for classification

Report the learning curves (training and validation accuracy vs number of epochs)

In [44]:
# TODO : Initialize the model and cast to correct device
model_ShallowConv = ShallowConvnet(input_channels = 3, num_classes = 10)
model_ShallowConv.to(device)

# TODO : Initialize the criterion
criterion = F.cross_entropy

# TODO : Initialize the SGD optimizer with lr 1e-3
optimizer = optim.SGD(model_ShallowConv.parameters(), lr=1e-3, momentum=0.5)

# TODO : Run the training loop using this model
train_losses, train_accuracies, val_losses, val_accuracies = train_loop(model_ShallowConv, criterion, optimizer, train_loader, val_loader)



[EPOCH]: 0, [TRAIN LOSS]: 2.193291, [TRAIN ACCURACY]: 17.422
[EPOCH]: 0, [VAL LOSS]: 2.184559, [VAL ACCURACY]: 23.940 

[EPOCH]: 1, [TRAIN LOSS]: 2.080832, [TRAIN ACCURACY]: 26.288
[EPOCH]: 1, [VAL LOSS]: 2.025002, [VAL ACCURACY]: 28.730 

[EPOCH]: 2, [TRAIN LOSS]: 2.037242, [TRAIN ACCURACY]: 29.545
[EPOCH]: 2, [VAL LOSS]: 1.875748, [VAL ACCURACY]: 30.960 



KeyboardInterrupt: ignored

In [ ]:
plt.plot(np.arange(1, 15651), train_accuracies)
plt.plot(np.arange(1, 3951), val_accuracies)

In [ ]:
train_loader.

### Q2.4 Simple convolution network - 

Design a convolutional neural network with the following specification: 
For each convolution layer, use appropriate padding such that it maintains the resolution of the image. The resolution should be changing only when you introduce maxpooling layers. Each convolution layer should be followed by a relu non-linearity. The first two blocks containing 3 convolutional layers are each followed by a maxpooling layer that halves the resolution of the image. After the third block, use maxpooling to get a resolution of 1 X 1. Finally, apply a linear transformation to project to the number of classes. 

Structure of the convolution layers of the model:
1. Number of input channels to the model = 3
2. First convolution layer of kernel size 5 with filter size 64 and padding such that it maintains the resolution of the image.
3. Followed by a block of 3 convolution layers of kernel size 3, filter size 64 and padding such that it maintains the resolution of the image.
5. Followed by 3 convolution layers of kernel size 3, filter size 128 and padding such that it maintains the resolution of the image.
6. Followed by 3 convolution layers of kernel size 3, filter size 256 and padding such that it maintains the resolution of the image. 




In [59]:
# Use the description of the structure of the model and the hints given below 

class SimpleConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleConvnet, self).__init__()
        
        # TODO
        self.network = nn.Sequential(
            # first layer, resolution = 32
            nn.Conv2d(input_channels, out_channels=64, kernel_size=5, padding=2),
            nn.ReLU(),
            # first block, resolution = 32
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),

            # first maxpooling, resolution = 16
            nn.MaxPool2d(kernel_size=2,stride=2),

            # second block, resolution = 32
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=9),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),

            # second maxpooling, resolution = 16
            nn.MaxPool2d(kernel_size=2, stride=2),

            # # third block, resolution = 32
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=9),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),

            # final maxpooling, output size reduced to 1
            nn.MaxPool2d(kernel_size=32),
            View((-1,256)),
            nn.Linear(256, num_classes)
      )
    
    def forward(self, x):
       
        # TODO 
        output = self.network(x)
        return output
        

In [60]:
# TODO : Initialize the model and cast to correct device
model_SimpleConv = SimpleConvnet(input_channels=3, num_classes=10)
model_SimpleConv.to(device)

# TODO : Initialize the criterion 
criterion = F.cross_entropy
# TODO : Initialize the optimizer 
optimizer = optim.SGD(model_SimpleConv.parameters(), lr=1e-3, momentum=0.5)
# TODO : Run the training loop using this model
train_losses_SimpleConv, train_accuracies_SimpleConv, val_losses_SimpleConv, val_accuracies_SimpleConv = train_loop(model_SimpleConv, criterion, optimizer, train_loader, val_loader)


[EPOCH]: 0, [TRAIN LOSS]: 2.302330, [TRAIN ACCURACY]: 10.055
[EPOCH]: 0, [VAL LOSS]: 2.311045, [VAL ACCURACY]: 9.780 

[EPOCH]: 1, [TRAIN LOSS]: 2.298062, [TRAIN ACCURACY]: 10.055
[EPOCH]: 1, [VAL LOSS]: 2.310381, [VAL ACCURACY]: 9.780 

[EPOCH]: 2, [TRAIN LOSS]: 2.302732, [TRAIN ACCURACY]: 10.055
[EPOCH]: 2, [VAL LOSS]: 2.309789, [VAL ACCURACY]: 9.780 

[EPOCH]: 3, [TRAIN LOSS]: 2.305752, [TRAIN ACCURACY]: 10.055
[EPOCH]: 3, [VAL LOSS]: 2.309226, [VAL ACCURACY]: 9.780 

[EPOCH]: 4, [TRAIN LOSS]: 2.302088, [TRAIN ACCURACY]: 10.055
[EPOCH]: 4, [VAL LOSS]: 2.308709, [VAL ACCURACY]: 9.780 

[EPOCH]: 5, [TRAIN LOSS]: 2.306312, [TRAIN ACCURACY]: 10.055
[EPOCH]: 5, [VAL LOSS]: 2.308244, [VAL ACCURACY]: 9.780 



### Q 2.5 Report results of training using SGD optimizer for both ShallowNet and SimpleConvnet. What do you observe?

The ShallowNet does learn, while the SimpleConvnet doesn't learn using this architecture.

### Q 2.6 Add batch normalization

#### Q2.6 a After each relu layer, add a batch normalization layer to the network SimpleConvnet you created above

In [13]:
class SimpleConvnet2(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleConvnet2, self).__init__()
        
        # TODO
        self.network = nn.Sequential(
            # first layer, resolution = 32
            nn.Conv2d(input_channels, out_channels=64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # first block, resolution = 32
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),

            # first maxpooling, resolution = 16
            nn.MaxPool2d(kernel_size=2,stride=2),

            # second block, resolution = 32
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=9),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),

            # second maxpooling, resolution = 16
            nn.MaxPool2d(kernel_size=2, stride=2),

            # # third block, resolution = 32
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=9),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),

            # final maxpooling, output size reduced to 1
            nn.MaxPool2d(kernel_size=32),
            View((-1,256)),
            nn.Linear(256, num_classes)
      )
    def forward(self, x):
       
       # TODO
        output = self.network(x)
        return output
        

In [14]:
model_SimpleConv2 = SimpleConvnet2(input_channels=3, num_classes=10)
model_SimpleConv2.to(device)

# TODO : Initialize the criterion 
criterion = F.cross_entropy
# TODO : Initialize the optimizer 
optimizer = optim.SGD(model_SimpleConv2.parameters(), lr=1e-3, momentum=0.5)
# TODO : Run the training loop using this model
train_losses_SimpleConv2, train_accuracies_SimpleCon2v, val_losses_SimpleConv2, val_accuracies_SimpleConv2 = train_loop(model_SimpleConv2, criterion, optimizer, train_loader, val_loader)


[EPOCH]: 0, [TRAIN LOSS]: 1.209702, [TRAIN ACCURACY]: 39.977
[EPOCH]: 0, [VAL LOSS]: 0.903356, [VAL ACCURACY]: 49.700 

[EPOCH]: 1, [TRAIN LOSS]: 1.386695, [TRAIN ACCURACY]: 53.877
[EPOCH]: 1, [VAL LOSS]: 0.812729, [VAL ACCURACY]: 54.680 

[EPOCH]: 2, [TRAIN LOSS]: 1.005105, [TRAIN ACCURACY]: 60.900
[EPOCH]: 2, [VAL LOSS]: 0.949280, [VAL ACCURACY]: 57.130 

[EPOCH]: 3, [TRAIN LOSS]: 0.990089, [TRAIN ACCURACY]: 66.830
[EPOCH]: 3, [VAL LOSS]: 0.716029, [VAL ACCURACY]: 61.920 

[EPOCH]: 4, [TRAIN LOSS]: 1.019390, [TRAIN ACCURACY]: 71.050
[EPOCH]: 4, [VAL LOSS]: 0.692875, [VAL ACCURACY]: 62.880 

[EPOCH]: 5, [TRAIN LOSS]: 0.590241, [TRAIN ACCURACY]: 74.785
[EPOCH]: 5, [VAL LOSS]: 0.619710, [VAL ACCURACY]: 64.470 

[EPOCH]: 6, [TRAIN LOSS]: 0.647320, [TRAIN ACCURACY]: 78.748
[EPOCH]: 6, [VAL LOSS]: 0.746756, [VAL ACCURACY]: 62.840 

[EPOCH]: 7, [TRAIN LOSS]: 0.489165, [TRAIN ACCURACY]: 81.710
[EPOCH]: 7, [VAL LOSS]: 0.702491, [VAL ACCURACY]: 67.670 

[EPOCH]: 8, [TRAIN LOSS]: 0.498233, [TRA

#### Q2.6 b Plot the the training curves (training loss vs \# epochs, training accuracy vs # epochs) using SGD (lr 1e-3) with and without batch normalization. Comment on the difference. 

#### Q2.6 c. Try running the same two networks with an Adam optimizer (lr 1e-4). Plot the the training curves (training loss vs \# epochs, training accuracy vs # epochs) with and without batch normalization. Comment on the difference. 

#### Q2.6 d Once you choose an optimizer and see that it does train, make sure your model has enough capacity by overfitting on one batch of the data set. You should be able to get 100% train accuracy. 


### Q 2.7 Add residual connections

#### Residual connections help stabilise training and lead to faster convergence. In this question you will introduce residual connections into the SimpleConvnet2 model that you built above. 

We will add residual connections after each block of 3 convolutional layers. Lets consider the first block of three convolutional layers. The input to this block, the so called residual, is added to the output of the block before the final batch normalization layer of that block. 


IMP NOTE: You will notice that the number of filters of these two summands are not the same. For this, you will need to use a convolution layer on the residual component, which changes the number of filters while keeping the rest of the dimensions the same. This can be achieved with a careful selection of the input_channels, output_channels, kernel_size and padding parameters. 

In [ ]:
class ResidualConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(ResidualConvnet, self).__init__()
        
        # TODO
    
    def forward(self, x):
        
        # TODO 
        return output
        

In [ ]:
# TODO : Initialize the model and cast to correct device

# TODO : Initialize the criterion 

# TODO : Initialize the optimizer 

# TODO : Run the training loop using this model

### Q 2.8 Plot the training curves with and without the residual connection. Comment on the difference. 

### Q2.9 Reducing overfiting 



In the previous questions, you might have observed that there is a large difference between the training and validation losses. This is a sign that the model is overfitting. One way to combat this is by adding random transformations to the input data to make your model more robust and prevent it from memorizing the input data. 

Torchvision provides several transforms that you can readily apply to your data. Experiment with adding a few transforms and report your results in terms of learning curves to see if the gap between the training and validation loss reduces and try to achieve better perfomance on the validation set. 

In [ ]:
test_transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         [0.4373, 0.4434, 0.4725],
                                         [0.1201, 0.1231, 0.1052])
                                     ])

train_transform = transforms.Compose([
                                      # TODO: Add more transforms here 
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.4373, 0.4434, 0.4725],
                                          [0.1201, 0.1231, 0.1052])
                                      ])


In [ ]:
train_dataset_w_transform  = MapDataset(train_dataset, train_transform)
val_dataset_w_transform = MapDataset(val_dataset, test_transform)
test_dataset_w_transform = MapDataset(test_dataset, test_transform)

In [ ]:
bs = 128
train_loader = DataLoader(train_dataset_w_transform, batch_size=bs, shuffle=True, drop_last=False)
val_loader = DataLoader(val_dataset_w_transform, batch_size=bs, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False, drop_last=False)

#### Use the residual network that you built above and use data augmentation to reduce the overfitting 

In [ ]:
# TODO : Initialize the model and cast to correct device

# TODO : Initialize the criterion 

# TODO : Initialize the optimizer 

# TODO : Run the training loop using this model

### Q 2.10  Effect of learning rate decay 

#### Here you need to change the training loop to have one additional operation - add the scheduler step at the end of each epoch. Experiment with different learning rate schedulers provided by pytorch. Report results using atleast StepLR. 

In [ ]:
from torch.optim.lr_scheduler import StepLR

In [ ]:
def train_loop2(model, criterion, optimizer, scheduler,  train_loader, val_loader):
    """
    Generic training loop

    Parameters
    ----------
    model : Object instance of your model class 
    criterion : Loss function 
    optimizer : Instance of optimizer class of your choice 
    scheduler : Instance of scheduler class of your choice 
    train_loader : Training data loader 
    val_loader : Validation data loader

    Returns
    -------
    train_losses : List with train loss on dataset per epoch
    train_accuracies : List with train accuracy on dataset per epoch
    val_losses : List with validation loss on dataset per epoch
    val_accuracies : List with validation accuracy on dataset per epoch

    """
    best_val = 0.0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    max_patience = 5
    patience_counter = 0

    # Training
    for t in tqdm(range(50)):

        # TODO : Set the model to train mode        

        # TODO: Loop over the training set 

            # TODO: Put the inputs and targets on the write device
            
            # TODO: Feed forward to get the logits

            # TODO: Compute the loss and accuracy

            # TODO: zero the gradients before running
            # the backward pass.

            # TODO: Backward pass to compute the gradient
            # of loss w.r.t our learnable params. 

            # TODO: Update params
            
            # TODO: Keep track of accuracy and loss

        
        # Switch the model to eval mode
        # TODO


        with torch.no_grad():
            # TODO: Loop over the validation set 
            
                # TODO: Put the inputs and targets on the write device
            
                # TODO: Feed forward to get the logits

                # TODO: Compute the loss and accuracy

                # TODO: Keep track of accuracy and loss

        if val_accuracies[-1] > best_val:
          best_val = val_accuracies[-1]
          patience_counter = 0

          # TODO: Save best model, optimizer, epoch_number
          
        else:
          patience_counter += 1    
          
          if patience_counter > max_patience: 
            break

        print("[EPOCH]: %i, [TRAIN LOSS]: %.6f, [TRAIN ACCURACY]: %.3f" % (t, train_losses[-1], train_accuracies[-1]))
        print("[EPOCH]: %i, [VAL LOSS]: %.6f, [VAL ACCURACY]: %.3f \n" % (t, val_losses[-1] ,val_accuracies[-1]))

        # TODO : scheduler step

    return train_losses, train_accuracies, val_losses, val_accuracies

In [ ]:
# TODO : Initialize the model and cast to correct device

# TODO : Initialize the criterion 

# TODO : Initialize the optimizer 

# TODO : Initialize the 

# TODO : Run the training loop using this model

### Q2.11 Hyper parameter tuning 


#### Experiment with a range of learning rates and optimizers, as well as the parameter in the learning rate scheduler for StepLR. Report the following plots: 

1. Learning curves (training and validation loss for 5 different learning rate with SGD optimizer)
2. Learning curves (training and validation loss for 5 different learning rate with Adam optimizer)
3. Learning curves (training and validation loss for 5 different gamma parameter for the StepLR)





### Q2.12 Load the model that gave you best validation accuracy and report results on the test set. 